In [2]:
import os
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, LSTM
from tensorflow.keras.utils import to_categorical

# Function to extract features from an audio file
def extract_features(file_name):
    try:
        audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
        mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        mfccs_processed = np.mean(mfccs.T, axis=0)
    except Exception as e:
        print("Error encountered while parsing file: ", file_name)
        return None 
    return mfccs_processed

# Load dataset and extract features
def load_data(dataset_path):
    labels = []
    features = []

    for folder in os.listdir(dataset_path):
        folder_path = os.path.join(dataset_path, folder)
        if os.path.isdir(folder_path):
            for file in os.listdir(folder_path):
                file_path = os.path.join(folder_path, file)
                data = extract_features(file_path)
                if data is not None:
                    features.append(data)
                    labels.append(folder)

    return np.array(features), labels

# Define dataset path
dataset_path = r"C:\Users\vinay\OneDrive\Desktop\FourClass"

# Load and preprocess data
features, labels = load_data(dataset_path)
labels = np.array(labels)
label_encoder = LabelEncoder()
labels_encoded = to_categorical(label_encoder.fit_transform(labels))

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

# Reshape data for LSTM layer
X_train = np.reshape(X_train, (X_train.shape[0], 1, X_train.shape[1]))
X_test = np.reshape(X_test, (X_test.shape[0], 1, X_test.shape[1]))

# Build model
model = Sequential()
model.add(LSTM(128, input_shape=(1, 40), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128))
model.add(Dropout(0.2))
model.add(Dense(labels_encoded.shape[1], activation='softmax'))

# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))

# Evaluate model
test_loss, test_accuracy = model.evaluate(X_test, y_test)
print("Test accuracy: {:.2f}%".format(test_accuracy * 100))


Epoch 1/50
320/320 [==============================] - 6s 9ms/step - loss: 1.0716 - accuracy: 0.5420 - val_loss: 0.9341 - val_accuracy: 0.5920
Epoch 2/50
320/320 [==============================] - 1s 5ms/step - loss: 0.9323 - accuracy: 0.5971 - val_loss: 0.8731 - val_accuracy: 0.6257
Epoch 3/50
320/320 [==============================] - 1s 5ms/step - loss: 0.8905 - accuracy: 0.6137 - val_loss: 0.8553 - val_accuracy: 0.6257
Epoch 4/50
320/320 [==============================] - 1s 5ms/step - loss: 0.8664 - accuracy: 0.6152 - val_loss: 0.8462 - val_accuracy: 0.6280
Epoch 5/50
320/320 [==============================] - 1s 5ms/step - loss: 0.8479 - accuracy: 0.6219 - val_loss: 0.8138 - val_accuracy: 0.6429
Epoch 6/50
320/320 [==============================] - 1s 5ms/step - loss: 0.8302 - accuracy: 0.6316 - val_loss: 0.8419 - val_accuracy: 0.6453
Epoch 7/50
320/320 [==============================] - 2s 5ms/step - loss: 0.8248 - accuracy: 0.6357 - val_loss: 0.7957 - val_accuracy: 0.6519
Epoch 

In [4]:
# import pickle

# # Save the trained model
# with open('fourClass.pkl', 'wb') as file:
    # pickle.dump(model, file)
model.save('fourclass.h5')

In [12]:
# import os
# import numpy as np
# import librosa
# import matplotlib.pyplot as plt
# from audiomentations import Compose, AddGaussianNoise, PitchShift, TimeStretch
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Flatten, Dropout
# from tensorflow.keras.utils import to_categorical

# # Data Augmentation
# augment = Compose([
#     AddGaussianNoise(min_amplitude=0.001, max_amplitude=0.02, p=0.5),
#     PitchShift(min_semitones=-4, max_semitones=4, p=0.5),
#     TimeStretch(min_rate=0.8, max_rate=1.25, p=0.5),
# ])

# # Function to create a spectrogram
# def create_spectrogram(audio, sample_rate):
#     stft = librosa.stft(audio)
#     spectrogram = librosa.amplitude_to_db(abs(stft))
#     return spectrogram

# # Feature extraction
# def extract_features(file_name, augment=None, max_pad_len=174):
#     try:
#         audio, sample_rate = librosa.load(file_name, res_type='kaiser_fast')
#         if augment is not None:
#             audio = augment(samples=audio, sample_rate=sample_rate)
#         mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
        
#         # Padding or truncating each MFCC to a fixed length
#         pad_width = max_pad_len - mfccs.shape[1]
#         if pad_width < 0:  # Truncate
#             mfccs = mfccs[:, :max_pad_len]
#         else:  # Pad
#             mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

#     except Exception as e:
#         print("Error encountered while parsing file: ", e)
#         return None 
#     return mfccs


# # Load dataset and extract features
# def load_data(dataset_path, augment=None):
#     labels = []
#     features = []

#     for folder in os.listdir(dataset_path):
#         folder_path = os.path.join(dataset_path, folder)
#         for file in os.listdir(folder_path):
#             file_path = os.path.join(folder_path, file)
#             data = extract_features(file_path, augment)
#             if data is not None:
#                 features.append(data)
#                 labels.append(folder)

#     return np.array(features), labels

# # Define dataset path
# dataset_path = r"C:\Users\vinay\OneDrive\Desktop\FourClass"

# # Load and preprocess data
# features, labels = load_data(dataset_path, augment=augment)
# labels = np.array(labels)
# label_encoder = LabelEncoder()
# labels_encoded = to_categorical(label_encoder.fit_transform(labels))

# # Split dataset
# X_train, X_test, y_train, y_test = train_test_split(features, labels_encoded, test_size=0.2, random_state=42)

# # CNN Model
# model = Sequential()
# model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
# model.add(MaxPooling2D((2, 2)))
# model.add(Conv2D(64, (3, 3), activation='relu'))
# model.add(MaxPooling2D((2, 2)))
# model.add(Flatten())
# model.add(Dense(64, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(labels_encoded.shape[1], activation='softmax'))

# # Compile model
# model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# # Train model
# model.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

# # Evaluate model
# test_loss, test_accuracy = model.evaluate(X_test, y_test)
# print("Test accuracy: {:.2f}%".format(test_accuracy * 100))


Epoch 1/100
320/320 [==============================] - 203s 630ms/step - loss: 1.4813 - accuracy: 0.4901 - val_loss: 0.9913 - val_accuracy: 0.5740
Epoch 2/100
320/320 [==============================] - 202s 631ms/step - loss: 1.0033 - accuracy: 0.5722 - val_loss: 0.9546 - val_accuracy: 0.5877
Epoch 3/100
320/320 [==============================] - 201s 629ms/step - loss: 0.9401 - accuracy: 0.5920 - val_loss: 0.8887 - val_accuracy: 0.5971
Epoch 4/100
320/320 [==============================] - 203s 633ms/step - loss: 0.9000 - accuracy: 0.6033 - val_loss: 0.8875 - val_accuracy: 0.5951
Epoch 5/100
320/320 [==============================] - 202s 632ms/step - loss: 0.8713 - accuracy: 0.6122 - val_loss: 0.8912 - val_accuracy: 0.6053
Epoch 6/100
320/320 [==============================] - 203s 633ms/step - loss: 0.8406 - accuracy: 0.6202 - val_loss: 0.8836 - val_accuracy: 0.6010
Epoch 7/100
320/320 [==============================] - 203s 633ms/step - loss: 0.8047 - accuracy: 0.6381 - val_loss: 0

In [ ]:
# import pickle

# # Save the trained model
# with open('emo2.pkl', 'wb') as file:
#     pickle.dump(model, file)

In [2]:
import keras
print(keras.__version__)


2.14.0


In [3]:
import librosa
print(librosa.__version__)

0.10.1


In [6]:
import tensorflow as tf
print(tf.__version__)


2.14.0


In [7]:
import sklearn
print(sklearn.__version__)


1.3.1


In [8]:
import pkg_resources

installed_packages = pkg_resources.working_set
installed_packages_list = sorted(["{}=={}".format(i.key, i.version) for i in installed_packages])

for package in installed_packages_list:
    print(package)


absl-py==2.0.0
anyio==4.0.0
argon2-cffi-bindings==21.2.0
argon2-cffi==23.1.0
arrow==1.3.0
asttokens==2.4.0
astunparse==1.6.3
async-lru==2.0.4
attrs==23.1.0
audiomentations==0.34.1
audioread==3.0.1
axios==0.4.0
babel==2.13.0
backcall==0.2.0
beautifulsoup4==4.12.2
bleach==6.1.0
blinker==1.7.0
cachetools==5.3.1
certifi==2023.7.22
cffi==1.16.0
charset-normalizer==3.3.0
click==8.1.7
colorama==0.4.6
colorlog==6.8.2
comm==0.1.4
contourpy==1.2.0
cycler==0.12.1
debugpy==1.8.0
decorator==5.1.1
defusedxml==0.7.1
exceptiongroup==1.1.3
executing==2.0.0
fastjsonschema==2.18.1
ffmpeg-python==0.2.0
filelock==3.13.1
flask-cors==4.0.0
flask==3.0.2
flatbuffers==23.5.26
fonttools==4.44.3
fqdn==1.5.1
fsspec==2023.10.0
future==0.18.3
gast==0.5.4
google-auth-oauthlib==1.0.0
google-auth==2.23.2
google-pasta==0.2.0
grpcio==1.59.0
h5py==3.9.0
idna==3.4
importlib-metadata==6.8.0
importlib-resources==6.1.1
install==1.3.5
ipykernel==6.25.2
ipython-genutils==0.2.0
ipython==8.16.1
ipywidgets==8.1.1
isoduration==20.1

C:\Users\vinay\AppData\Local\Temp\ipykernel_1992\4066627098.py:1: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  import pkg_resources
